In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-ml-images/finaltest.csv
/kaggle/input/amazon-ml-images/file1.csv
/kaggle/input/amazon-ml-images/file2.csv
/kaggle/input/amazon-ml-images/file4.csv
/kaggle/input/amazon-ml-images/file3.csv
/kaggle/input/amazon-ml-images/dataset.csv


In [85]:
import pandas as pd
import numpy as np
import os

import string
from tqdm import tqdm
import re

In [86]:
megaframe=pd.read_csv('/kaggle/input/amazon-ml-images/finaltest.csv')
megaframe

,Unnamed: 0,index,image_link,group_id,entity_name,textfromimg
0,0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839.0,height,S6ban 36in - ?ug 91 bauMn I5eii
1,1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,width,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
2,2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,height,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
3,3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,depth,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
4,4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578.0,depth,"Size Width Length One Size 10.5Ocm/4.13"" 9Ocm/..."
...,...,...,...,...,...,...
131183,541,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522.0,maximum_weight_recommendation,"Tite 1,500 LB weight capacity Foren 0o Pnr Ptrran"
131184,542,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688.0,item_weight,BUILT-IN HANDLES COMES WITH DURABLE HANDLES FO...
131185,543,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688.0,maximum_weight_recommendation,BUILT-IN HANDLES COMES WITH DURABLE HANDLES FO...
131186,544,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009.0,item_weight,DoCMAME Jossie HuMannaMe Victor PHONENUMBER (6...


In [87]:
megaframe = megaframe.drop(['Unnamed: 0'], axis=1)
megaframe

,index,image_link,group_id,entity_name,textfromimg
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839.0,height,S6ban 36in - ?ug 91 bauMn I5eii
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,width,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,height,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,depth,"Size Width Length One Size 42cm/16.54"" 20Ocm/7..."
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578.0,depth,"Size Width Length One Size 10.5Ocm/4.13"" 9Ocm/..."
...,...,...,...,...,...
131183,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522.0,maximum_weight_recommendation,"Tite 1,500 LB weight capacity Foren 0o Pnr Ptrran"
131184,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688.0,item_weight,BUILT-IN HANDLES COMES WITH DURABLE HANDLES FO...
131185,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688.0,maximum_weight_recommendation,BUILT-IN HANDLES COMES WITH DURABLE HANDLES FO...
131186,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009.0,item_weight,DoCMAME Jossie HuMannaMe Victor PHONENUMBER (6...


In [88]:
unit_abbreviation_map = {
    'g': 'gram',
    'grams': 'gram',
    'kg': 'kilogram',
    'mg': 'milligram',
    'cup': 'cup',
    'mg': 'milligram',
    'kg': 'kilogram',
    'oz': 'ounce',
    'gallon': 'gallon',
    'v': 'volt',
    'volt': 'volt',
    'w': 'watt',
    'watt': 'watt',
    'lb': 'pound','pound': 'pound',
    'ml': 'millilitre',
    'millilitre': 'millilitre',
    'ft': 'foot',
    'foot': 'foot',
    'ton': 'ton',
    'dl': 'decilitre',
    'decilitre': 'decilitre',
    'in': 'inch',
    'inch': 'inch',
    'l': 'litre',
    'litre': 'litre',
    'mcg': 'microgram',
    'microgram': 'microgram',
    'cm': 'centimetre',
    'centimetre': 'centimetre',
    'qt': 'quart',
    'quart': 'quart',
    'hp': 'horsepower',
    'horsepower': 'horsepower',
    'kw': 'kilowatt',
    'kilowatt': 'kilowatt'
}

In [90]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','m','cm','in','ft'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton',
        'g',
        'kg',
        'mg','lb'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton',
        'kg','g','mg'},
    'voltage': {'kilovolt', 'millivolt', 'volt','v','kv','mv'},
    'wattage': {'kilowatt', 'watt','w','kw'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart','ml'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}
allowed_units

{'centilitre',
 'centimetre',
 'cm',
 'cubic foot',
 'cubic inch',
 'cup',
 'decilitre',
 'fluid ounce',
 'foot',
 'ft',
 'g',
 'gallon',
 'gram',
 'imperial gallon',
 'in',
 'inch',
 'kg',
 'kilogram',
 'kilovolt',
 'kilowatt',
 'kv',
 'kw',
 'lb',
 'litre',
 'm',
 'metre',
 'mg',
 'microgram',
 'microlitre',
 'milligram',
 'millilitre',
 'millimetre',
 'millivolt',
 'ml',
 'mv',
 'ounce',
 'pint',
 'pound',
 'quart',
 'ton',
 'v',
 'volt',
 'w',
 'watt',
 'yard'}

In [91]:
megaframe['processed_text'] = megaframe['textfromimg'].str.lower()
megaframe.head()

,index,image_link,group_id,entity_name,textfromimg,processed_text
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839.0,height,S6ban 36in - ?ug 91 bauMn I5eii,s6ban 36in - ?ug 91 baumn i5eii
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,width,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...","size width length one size 42cm/16.54"" 20ocm/7..."
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,height,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...","size width length one size 42cm/16.54"" 20ocm/7..."
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,depth,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...","size width length one size 42cm/16.54"" 20ocm/7..."
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578.0,depth,"Size Width Length One Size 10.5Ocm/4.13"" 9Ocm/...","size width length one size 10.5ocm/4.13"" 9ocm/..."


In [92]:
def convert_floats_to_strings(megaframe):
    def is_float(word):
        try:
            float(word)
            return True
        except ValueError:
            return False

    # Iterate over the 'processed_text' column
    for index, row in megaframe.iterrows():
        text = row['processed_text']
        words = text.split()  # Split the text into individual words

        # Replace floats with strings
        words = [str(float(word)) if is_float(word) else word for word in words]

        # Join the words back into a single string and update the DataFrame
        megaframe.at[index, 'processed_text'] = ' '.join(words)

    return megaframe


In [93]:
megaframe['processed_text'] = megaframe['processed_text'].astype(str)
megaframe['processed_text'] = megaframe['processed_text'].str.replace('o', '0').str.replace('z', '2').str.replace('"','in')

In [94]:
def extract_numeric_units(text, allowed_units):
    # Regular expression to match patterns like 2.4g, 5.8g, etc.
    pattern = re.compile(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+|")(?:\/(\d+(?:\.\d+)?)\s*([a-zA-Z]+|"))?')
    
    # Find all matches
    matches = pattern.findall(text)
    
    # Filter matches based on the allowed units for this specific entity
    extracted = []
    for match in matches:
        number, unit, number2, unit2 = match
        
        # Check if the first unit is in the allowed units for the entity
        if unit in allowed_units:
            extracted.append(f"{number} {unit}")
        
        # If a second number/unit pair exists, check it as well
        if number2 and unit2 in allowed_units:
            extracted.append(f"{number2} {unit2}")
    
    return extracted

# Initialize the new column in your DataFrame
megaframe['extracted_values'] = [[] for _ in range(len(megaframe))]

# Iterate through each row and extract values based on the entity_name
for index, row in megaframe.iterrows():
    text = row['processed_text']
    entity_name = row['entity_name']
    
    # Get the allowed units for the current entity_name from entity_unit_map
    allowed_units_for_entity = entity_unit_map.get(entity_name, set())  # Default to an empty set if no mapping exists
    
    # Extract numeric-unit pairs based on the allowed units for the specific entity
    extracted = extract_numeric_units(text, allowed_units_for_entity)
    
    # Append extracted values to the new column
    megaframe.at[index, 'extracted_values'] = extracted

In [95]:
megaframe[['entity_name','extracted_values']].head(20)

,entity_name,extracted_values
0,height,[36 in]
1,width,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
2,height,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
3,depth,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
4,depth,"[10.50 cm, 4.13 in, 90 cm, 35.43 in]"
5,height,"[10.50 cm, 4.13 in, 90 cm, 35.43 in]"
6,width,"[10.50 cm, 4.13 in, 90 cm, 35.43 in]"
7,height,[0 m]
8,width,"[40 cm, 30 cm, 15 cm]"
9,height,"[40 cm, 30 cm, 15 cm]"


In [96]:
def count_empty_extracted_values(megaframe):
    empty_count = 0
    for values in megaframe['extracted_values']:
        if not values:  # Check if the list is empty
            empty_count += 1
    return empty_count

# Usage
empty_lists_count = count_empty_extracted_values(megaframe)
print(f"Number of empty lists in 'extracted_values': {empty_lists_count}")

Number of empty lists in 'extracted_values': 39581


In [97]:
def filter_non_empty_extracted_values(megaframe):
    # Filter rows where 'extracted_values' is not empty
    filtered_megaframe = megaframe[megaframe['extracted_values'].apply(bool)]
    return filtered_megaframe

non_empty_rows = filter_non_empty_extracted_values(megaframe)
non_empty_rows

,index,image_link,group_id,entity_name,textfromimg,processed_text,extracted_values
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839.0,height,S6ban 36in - ?ug 91 bauMn I5eii,s6ban 36in - ?ug 91 baumn i5eii,[36 in]
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,width,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...",si2e width length 0ne si2e 42cm/16.54in 200cm/...,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,height,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...",si2e width length 0ne si2e 42cm/16.54in 200cm/...,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578.0,depth,"Size Width Length One Size 42cm/16.54"" 20Ocm/7...",si2e width length 0ne si2e 42cm/16.54in 200cm/...,"[42 cm, 16.54 in, 200 cm, 78.74 in]"
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578.0,depth,"Size Width Length One Size 10.5Ocm/4.13"" 9Ocm/...",si2e width length 0ne si2e 10.50cm/4.13in 90cm...,"[10.50 cm, 4.13 in, 90 cm, 35.43 in]"
...,...,...,...,...,...,...,...
131174,131274,https://m.media-amazon.com/images/I/A1brlRJvdn...,299791.0,item_weight,Come with 24 Inches High Quality Durable Stain...,c0me with 24 inches high quality durable stain...,[36 g]
131175,131275,https://m.media-amazon.com/images/I/A1cAJrp-T5...,636545.0,item_weight,"Iilk McCormick"" ONE SHEET PAN GLAZED SALMON 8 ...",iilk mcc0rmickin 0ne sheet pan gla2ed salm0n 8...,"[0 g, 2 g, 72.07881 g, 480 mg]"
131176,131276,https://m.media-amazon.com/images/I/A1eo0U2pp+...,240413.0,voltage,1ñ 1 326B PC4.19200 ECC REG 2RX' CL17 1.2v MTA...,1ñ 1 326b pc4.19200 ecc reg 2rx' cl17 1.2v mta...,"[1.2 v, 1.2 v]"
131178,131278,https://m.media-amazon.com/images/I/A1hB6lQgjt...,748919.0,item_weight,KEEP FROZEN UNTIL READY TO USE. Crushed GARLIC...,keep fr02en until ready t0 use. crushed garlic...,[80 g]


In [98]:
# Function to map abbreviated units to full names
def map_units(value, entity_name):
    # Extract the number and unit from the value (e.g., '500g', '0.729 g')
    match = re.match(r"([0-9.]+)\s*([a-zA-Z]+)", value)
    if match:
        number, unit = match.groups()
        # Convert the number to float to maintain precision
        number_float = float(number)
        # Map unit using the abbreviation map
        full_unit = unit_abbreviation_map.get(unit, unit)  # Default to the same unit if not found in map
        return f"{number_float} {full_unit}"
    return value

# Function to find the most frequent value and map its unit to full name
# Function to compare the first and last value in the list and return the greater one
def get_predicted_value(row):
    entity_name = row['entity_name']
    values = row['extracted_values']
    
    # Check if extracted_values is None or empty
    if values is None or len(values) == 0:
        return None
    
    # Convert all values to their full unit names
    full_values = [map_units(value, entity_name) for value in values]
    
    # Extract the numeric part of the first and last value
    first_value = float(re.match(r"([0-9.]+)", full_values[0]).group(1))
    last_value = float(re.match(r"([0-9.]+)", full_values[-1]).group(1))
    
    # Compare and return the greater value (with its full unit)
    if first_value >= last_value:
        return full_values[0]
    else:
        return full_values[-1]


In [101]:
megaframe['prediction'] = megaframe.apply(get_predicted_value, axis=1)

print(megaframe[['entity_name','prediction']].head(20))

                      entity_name       prediction
0                          height        36.0 inch
1                           width       78.74 inch
2                          height       78.74 inch
3                           depth       78.74 inch
4                           depth       35.43 inch
5                          height       35.43 inch
6                           width       35.43 inch
7                          height            0.0 m
8                           width  40.0 centimetre
9                          height  40.0 centimetre
10                    item_weight             None
11                          width  41.6 centimetre
12                          depth  10.0 centimetre
13                         height  10.0 centimetre
14                          width  10.0 centimetre
15                    item_weight             None
16  maximum_weight_recommendation             None
17                    item_weight             None
18                        watta

In [100]:
output=megaframe[['index','prediction']]
output.to_csv('test_out_new.csv', index=False)